In [5]:
#import packages
import numpy as np
import pandas as pd
from keras import Sequential
from keras.models import load_model
from keras.layers import Dense, Activation, LSTM
import time
import datetime
import queue

Using TensorFlow backend.


In [6]:
from PyQt5.QtCore import pyqtSlot, pyqtSignal, QObject
import win32com.client
class Eventhandler:
    
    def a(self):
        print('a')
        
    @classmethod
    def get_instance(cls):
        cp_cur = win32com.client.DispatchWithEvents("dscbo1.StockCur", cls)
        return cp_cur

a = Eventhandler.get_instance()
a.a()

com_error: (-2147221164, '클래스가 등록되지 않았습니다.', None, None)

In [ ]:
#CREON API REALTIME
import sys
from PyQt5.QtCore import pyqtSlot, pyqtSignal, QObject
import win32com.client
class CpMarketEye:
    def Request(self, codes, rqField):
        objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
        bConnect = objCpCybos.IsConnect
        if(bConnect == 0):
            print("PLUS가 정상적으로 연결되지 않음.")
            return False
        objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        objRq.SetInputValue(0, rqField)
        objRq.SetInputValue(1, codes)
        #objRq.BlockRequest() #Block mode가 뭐임?
        rqStatus = objRq.GetDibStatus()
        rqRet = objRq.GetDibMsg1()
        print('통신상태' , rqStatus, rqRet)
        if rqStatus != 0:
            return False
        
        #종목하나만 넣을거지만, codes자료형이 배열로 되야하는지 아닌지 몰라서 일단 넣음
        cnt = objRq.GetHeaderValue(2)
        for i in range(cnt):
            rpCode = objRq.GetDataValue(0, i)
            rpName = objRq.GetDataValue(1, i)
            rpTime = objRq.GetDataValue(2,i)
            rpCurPrice = objRq.GetDataValue(5,i)
            rpVol = objRq.GetDataValue()(6,i)
            print(rpCode, rpName, rpTime, rpCurPrice, rpVol)
        
        return True
    
class StockCur(QObject):
    receive_real_stockcur_signal = pyqtSignal(tuple)
    
    def set_params(self, client):
        self.client = client
    
    def get_real(self, stockcode):
        stockcode = 'A'+stockcode
        self.SetInputValue(0, stockcode)
        self.Subscribe()
    
    def stop_real(self, stockcode:str):
        stockcode = 'A'+stockcode
        self.SetInputValue(0, stockcode)
        self.Unsubscribe()
    
    def OnReceived(self):
        code = self.client.GetHeaderValue(0)
        name = self.client.GetHeaderValue(1)
        ttime = self.client.GetHeaderValue(18)
        price = self.client.GetHeaderValue(13)
        cVol = self.client.GetHeaderValue(17)
        buysell = str(self.client.GetHeaderValue(26)) # 호가방식 체결상태 1:매수, 2:매도
        
        #treshold 값을 시총을 이용해서 구해주는것도 나쁘지않을듯.
        if cVol*price < 1000000: #100만원 이하의 피라미 주문은 안받음.
            pass
        else:
            tick = (code, name, ttime, price, cVol, buysell)
            self.receive_real_stockcur_signal.emit(tick)
    
    @classmethod
    def get_instance(cls):
        cp_cur = win32com.client.DispatchWithEvents("dscbo1.StockCur", cls)
        return cp_cur
    
    
            

In [ ]:
#CREON API TRANSACTION
import win32com.client

#연결여부 체크
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
bConnect = ojbCpCybos.IsConnect
if (bConnect == 0):
    print("PLUS가 정상적으로 연결되지 않음.")
    exit()

#주문초기화
objTrade = win32com.client.Dispatch("CpTrade.CpTdUtil")
initCheck = objTrade.TradeInit(0)
if (initCheck != 0):
    print("주문 초기화 실패")
    exit()

acc = objTrade.AccountNum[0]
accFlag = objTrade.GoodsList(acc, 1)
print(acc, accFlag[0])
objStockOrder = win32com.client.Dispatch("CpTrade.CpTd0311")
objStockOrder.SetInputValue(0, "2") #2:매수 1:매도
objStockOrder.SetInputValue(1, acc)
objStockOrder.SetInputValue(2, accFlag[0])#두번째 인자값 : 1:주식, 2:선옵 16:eurex 64:해선
objStockOrder.SetInputValue(3, "A003540") #종목코드
objStockOrder.SetInputValue(4, 1) #매수수량
#objStockOrder.SetInputValue(5, 12400) #주문 단가 입력. 시장가에서 입력 필요X
objStockOrder.SetInputValue(7, "0") # 0:기본 1:IOC(미체결자동취소) 2:FOK(미체결수량 있으면 전체 취소, 체결안됨)
objStockOrder.SetInputValue(8, "03") # "01" 보통주문 / "03" 시장가주문

#매수 주문 요청
objStockOrder.BlockRequest()
rqStatus = objStockOrder.GetDibStatus()
rqRet = objStockOrder.GetDibMsg1()
print("통신상태", rqStatus, rqRet) # 통신상태 0 0040 모의투자 매수 주문이 완료되었습니다.(ordss.cststkord)
if rqStatus != 0:
    exit()


In [9]:
#HyperParameters

input_dim = 2 #입력하는 feature 개수
#time_interval = 8 #minute. n분 마다 n분 누적 신호 발생. 혹은 lstm cell에서 시계열의 갯수
min_transaction_amount = 1000000 # 100만원 미만이면 리시버에서 버퍼로 데이터 안보내게끔.
n_cell = 32 # LSTM셀의 개수
n_labels = 4 #모델의 결과 값 라벨 수 e.g. n=4, 라벨이 강하락, 약하락, 약상승, 강상승 신호 발생.
seq_length = 14 # 모델이 참고 할 이전 분봉의 개수. LSTM/RNN에서
lr = 0.003 # 학습률
keep_prob = 1.0 #dropout rate
n_stacked_layers = 1 # 모델의 레이어 개수
n_epochs = 1000 # 학습횟수. 얼리스타핑 넣을까말까
n_batch_size = 32 #학습 배치 사이즈
model_path = None

In [ ]:
# DataLoading on API
# API에서 필요한 데이터를 가져오는 부분.
# 트레이딩 알고리즘에 실시간으로 전송할 데이터를 가져오는 함수 필요
# 모델 학습 및 테스팅 데이터 가져오는 부분은 다른 코드에서 따로 가져오기...바랍니다... 아님 여기다 넣어도 되고..

class DataReciever:
    def __init__(self, stockCode, buffer):
        self.api = CreonAPI.get_instance()
        self.stockCode = stockCode
        self.buffer = buffer#??? 버퍼와 데이터리시버는 상호참조? 혹은 리시버 has 버퍼
    
    #데이터를 불러올 때마다 발생
    def on_LoadData(self):
        self.buffer.add(api.data???)#??? 이부분은 구체적으로 api가 어떻게 이벤트를 발생시키는지 모르겠네요 아직
        #버퍼로 데이터 전달할 때, 받아온 체결 데이터에서 threshold를 min_transaction_amount / 체결가 만큼 넣어주면 좋을거같아요.
        #이렇게 하면 100만원 이하로 구매한 매매체결은 버퍼에 안들어가서 노이즈를 줄일 수 있을것 같습니다.
        pass
    
    # 데이터 발생 시 마다 버퍼가 데이터를 자동으로 받음.
    #def emitData(self):

In [ ]:
# Data Preprocessing for Predicting
# 실제 트레이딩 시, API에서 받아온 실시간 데이터를 버퍼에 넣기 전까지 전처리 하는 과정
# 데이터가 버퍼에 들어가기 전에 여기서 다듬고 나서 보내준다.
# 데이터 리시버가 이 클래스를 소유하고, 다듬은 다음 버퍼로 쏴주면 될듯? 아니면 버퍼가 프리프로세서를 소유해도 될듯.

In [ ]:
# Data Preprocessing for Training
# API에서 받아온 대용량 데이터를 모델 학습을 위해 인풋과 라벨값이 있는 지도학습 데이터로 바꿔주는 부분은 다른 곳에서 직접 짜기.

In [ ]:
# Data Buffer
# API에서 받아온 데이터가 전처리 되어 저장되는 공간.
class DataBuffer:
    def __init__(self, stockcode):
        self.stockcode = stockcode
        self.q = queue.Queue(maxsize = seq_length)
        self.stockcur = StockCur.get_instance()
        self.stockcur.receive_real_stockcur_signal.connect(self.__receive_real_stockcur)
    
  
        
    def add(self,data):
        #여기다가 데이터 전처리 부분 넣을까말까
        
        if self.q.full():#q가 가득차면
            self.q.get()# 하나 꺼내고
            self.q.put(data)# 신규 데이터 넣고
        else:
            self.q.put(data)
            
    #모인 데이터를 df 형식으로 쏴준다.
    def convert_to_DF(self):
        return pd.DataFrame(list(self.q.queue))
    
    @pyqtSlot(tuple)
    def __receive_real_stockcur(self, tick: tuple):
        self.add(str(tick))
    
    if __name__ == "__main__":
        _ = Buffer("005930")
        _.subscribe()
    #Queue를 만들고 Queue가 갱신(새 데이터가 들어옴)될 때 마다 모델에 데이터 전송

In [ ]:
# Data Seperating ( Training / Validating / Testing)
# 학습용 데이터 전처리 과정에서 받아온 데이터를 트레이닝 밸리데이팅 테스팅 데이터로 나누는 부분

from sklearn.model_selection import train_test_split
testSize = 0.3 #전체 데이터에서 테스트 데이터의 크기
x_train, x_test, y_train, y_test = train_test_split(preprocessed_Data, 
                                                    preprocessed_Data['Label'], 
                                                    test_size = testSize, 
                                                    random_state=13)


In [149]:
np.asarray([np.asarray([[1,23,14,14,14,14,214,12,41,12,4,214,1,42]]).T,])

array([[[  1],
        [ 23],
        [ 14],
        [ 14],
        [ 14],
        [ 14],
        [214],
        [ 12],
        [ 41],
        [ 12],
        [  4],
        [214],
        [  1],
        [ 42]]])

In [131]:
np.asarray([np.asarray([[1,23,14,14,14,14,214,12,41,12,4,214,1,42]]).T,
            np.asarray([[1,23,14,14,14,14,214,12,41,12,4,214,1,42]]).T,
            np.asarray([[1,23,14,14,14,14,214,12,41,12,4,214,1,42]]).T,])

array([[[  1],
        [ 23],
        [ 14],
        [ 14],
        [ 14],
        [ 14],
        [214],
        [ 12],
        [ 41],
        [ 12],
        [  4],
        [214],
        [  1],
        [ 42]],

       [[  1],
        [ 23],
        [ 14],
        [ 14],
        [ 14],
        [ 14],
        [214],
        [ 12],
        [ 41],
        [ 12],
        [  4],
        [214],
        [  1],
        [ 42]],

       [[  1],
        [ 23],
        [ 14],
        [ 14],
        [ 14],
        [ 14],
        [214],
        [ 12],
        [ 41],
        [ 12],
        [  4],
        [214],
        [  1],
        [ 42]]])

In [ ]:
(데이터 샘플의 갯수, 시계열 길이, 피쳐의 갯수)

In [130]:
np.asarray(([1,2,3,4], [5,6,7,8]))

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [121]:
np.asarray([1,2,3,4])

array([1, 2, 3, 4])

In [127]:
np.asarray(([1,2,3,4],[1,2,3,4],[4,5,6,7]))

array([[1, 2, 3, 4],
       [1, 2, 3, 4],
       [4, 5, 6, 7]])

In [164]:
# Model Training & Saving
# 모델 생성 및 학습. 결과 저장

"""모델 라벨값은 총 4개. 0,1,2,3 으로 라벨링하고 
0(강 하락)에 매도, 1(약 하락), 2(약 상승)에 홀드or관망, 3(강 상승)에 매수"""

from keras import metrics

model = Sequential()
for i in range(n_stacked_layers):
    model.add(LSTM(n_cell, input_shape=(seq_length, input_dim), activation='sigmoid',weights=None))
model.add(Dense(4, activation='softmax'))

model.compile(loss='mse', optimizer='adam')
#어떤 loss func, optimizer를 써야할지, 메트릭스를 써야할지 모르겠다. 일단 나중에 생각하기.
history = model.fit(x = x_train, y = y_train, batch_size=n_batch_size, epochs = n_epochs, validation_split=0.3)
#밸리데이션셋은 이전 단계에서 햇음. 데이터를 트레인 테스트로 8:2로 나눈다음 8을 
#5.6 : 2.4로 트레인 : 밸릳 데이터로 나눔.

#모델 저장
model_path = f"./model_{datetime.datetime.now().strftime('%m%d_%H%M%S')}.h5"
model.save(model_path)
import matplotlib.pyplot as plt
plt.plot(history.losses)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

#model.evaluate()
#model.save()

NameError: name 'x_train' is not defined

In [ ]:
np.asarray([np.asarray([[1,23,14,14,14,14,214,12,41,12,4,214,1,42]]).T

In [150]:
model.predict(np.asarray([np.asarray([[1,23,14,14,14,14,214,12,41,12,4,214,1,42]]).T,]))

array([[4.0882558e-05, 3.9941187e-05, 7.8973483e-04, 9.9912947e-01]],
      dtype=float32)

In [151]:
a = [4.0882558e-05, 3.9941187e-05, 7.8973483e-04, 9.9912947e-01]

In [156]:
a= np.asarray(a)

In [163]:
a.round(3)

array([0.   , 0.   , 0.001, 0.999])

In [78]:
np.asarray(a).sum()

0.999999968365

In [ ]:
# Model Testing(생략가능)
# 테스팅 데이터로 모델 테스트.
# 목표 Loss 값이 될 때 까지 전 단계(생성/학습) 반복
# 한 칸 위 코드를 모듈화해서 하이퍼파라미터 그리드 서치 후 최고 점수 모델 저장.

In [ ]:
# Trading with Trained Model
# 모델이 발생시킨 신호를 판단하여 API에 매매 요청.
class Trader:
    def __init__(self, logger):
        self.logger = logger
    
    def buy(self,):
        self.logger.addLog("buy", n_buy, buy_price, account_balance)
        pass
        
    def sell(self,):
        pass
    
    def action(self, sign):
        if sign == 0:
            if 계좌에 포지션이 존재하는 경우:
                self.sell()
            elif 계좌에 포지션이 없는 경우:
                pass # 로그 찍기("매도신호 발생. 포지션 없음." 이런식으로)
        elif sign == 3:
            if 계좌에 현금이 있는 경우:
                self.buy()
            elif 계좌에 현금이 없는 경우:
                pass
            

In [ ]:
import sys
from PyQt5.QtWidgets import *
import win32com.client

# Trading with Trained Model
# 모델이 발생시킨 신호를 판단하여 API에 매매 요청.
class Trader:
    #CREON API TRANSACTION
    def stockorder(order):
        #연결여부 체크
        objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
        bConnect = ojbCpCybos.IsConnect
        if (bConnect == 0):
            print("PLUS가 정상적으로 연결되지 않음.")
            exit()

        #주문초기화
        objTrade = win32com.client.Dispatch("CpTrade.CpTdUtil")
        initCheck = objTrade.TradeInit(0)
        if (initCheck != 0):
            print("주문 초기화 실패")
            exit()

        acc = objTrade.AccountNum[0]
        accFlag = objTrade.GoodsList(acc, 1)
        print(acc, accFlag[0])
        objStockOrder = win32com.client.Dispatch("CpTrade.CpTd0311")
        objStockOrder.SetInputValue(0, order) #order 2:매수 1:매도
        objStockOrder.SetInputValue(1, acc)
        objStockOrder.SetInputValue(2, accFlag[0])#두번째 인자값 : 1:주식, 2:선옵 16:eurex 64:해선
        objStockOrder.SetInputValue(3, "A003540") #종목코드
        objStockOrder.SetInputValue(4, 1) #매수수량
        #objStockOrder.SetInputValue(5, 12400) #주문 단가 입력. 시장가에서 입력 필요X
        objStockOrder.SetInputValue(7, "0") # 0:기본 1:IOC(미체결자동취소) 2:FOK(미체결수량 있으면 전체 취소, 체결안됨)
        objStockOrder.SetInputValue(8, "03") # "01" 보통주문 / "03" 시장가주문

        #매수 주문 요청
        objStockOrder.BlockRequest()
        
        rqStatus = objStockOrder.GetDibStatus()
        rqRet = objStockOrder.GetDibMsg1()
        print("통신상태", rqStatus, rqRet) # 통신상태 0 0040 모의투자 매수 주문이 완료되었습니다.(ordss.cststkord)
        if rqStatus != 0:
            exit()
            
        #주문 내역
        order_num = objStockOrder.GetHeaderValue(8)              # 주문번호
        print("주문번호: ", order_num, " >> 주문 완료")
        
        #체결 내역
        conclusion = win32com.client.Dispatch("DsCbo1.CpConclusion")
        
        #conflag = self.client.GetHeaderValue(14)    # 체결 플래그
        ordered_num = conclusion.GetHeaderValue(5)      # 체결된 주문번호
        amount = conclusion.GetHeaderValue(3)        # 체결 수량
        price = conclusion.GetHeaderValue(4)         # 가격
        code = conclusion.GetHeaderValue(9)          # 종목코드
        
        if(order == "1"):
            print( "주문번호: ", ordered_num, ", 체결 수량: ", amount, ", 체결 가격: ", price, ", 종목코드: ", code , " >> 매도 완료")
        elif(order == "2"):
            print( "주문번호: ", ordered_num, ", 체결 수량: ", amount, ", 체결 가격: ", price, ", 종목코드: ", code , " >> 매수 완료")
            

         
    def action(self, sign):
        if sign == 0:
            if 계좌에 포지션이 존재하는 경우:
                self.stockorder("1")
            elif 계좌에 포지션이 없는 경우:
                print("매도신호 발생, 포지션 없음.")
                pass # 로그 찍기("매도신호 발생. 포지션 없음." 이런식으로)
        elif sign == 3:
            if 계좌에 현금이 있는 경우:
                self.stockorder("2")
            elif 계좌에 현금이 없는 경우:
                print("매수신호 발생, 잔액 부족.")
                pass
            

In [ ]:
# Logging
# 트레이딩 결과 로깅 및 기초통계량 작성
class Logger:
    def __init__(self):
        self.logs = []
    
    def addLog(self, *args)
        log = []
        for i in args:
            log.append(i)
        now = datetime.datetime.now()
        log = str(now) + str(log)
        self.logs.append(log)
        print(log)
    
    def saveLogs(self):
        with open(f"./logs_{datetime.datetime.now().strftime('%m%d_%H%M%S')}.txt", 'w') as f:
            for log in self.logs:
                f.write("%s\n" % log)
                  
    def statistics(self):
        pass

In [2]:
import pandas as pd
data = pd.read_csv('./temptemp.txt', sep="\)\(", header=None)
data

/Users/yongcheolkim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0,1,2,3,4,5,6,7,8,9,...,66571,66572,66573,66574,66575,66576,66577,66578,66579,66580
0,"('000660', '2020060490334', 91200, 91100, 9120...","'000660', '2020060490334', 91200, 91100, 91100...","'000660', '2020060490334', 91200, 91100, 91200...","'000660', '2020060490334', 91200, 91100, 91100...","'000660', '2020060490334', 91200, 91100, 91200...","'000660', '2020060490334', 91200, 91100, 91100...","'000660', '2020060490334', 91200, 91100, 91200...","'000660', '2020060490334', 91200, 91100, 91200...","'000660', '2020060490334', 91200, 91100, 91100...","'000660', '2020060490334', 91200, 91100, 91200...",...,"'000660', '20200604151921', 87800, 87700, 8780...","'000660', '20200604151921', 87800, 87700, 8780...","'000660', '20200604151921', 87800, 87700, 8770...","'000660', '20200604151922', 87800, 87700, 8780...","'000660', '20200604151922', 87800, 87700, 8780...","'000660', '20200604151922', 87800, 87700, 8770...","'000660', '20200604151922', 87800, 87700, 8780...","'000660', '20200604151922', 87800, 87700, 8770...","'000660', '20200604151922', 87800, 87700, 8780...","'000660', '20200604151923', 87800, 87700, 8780..."


In [19]:
data[0][1]

"'000660', '2020060490334', 91200, 91100, 91100, '2', '2', '-', 909, 519131, 151521, 367163, 4700286, '2'"

In [34]:
data[0][1][41:47]

'91100,'

In [38]:
data[0][1][59:61]

"-'"

In [42]:
data[0][1][63:67]

'909,'

In [14]:
data = data.T[1:-1]

In [45]:
for i in range(data.shape[0]):
    data['price'] = data[0][41:47]

In [46]:
data

,0,price
1,"'000660', '2020060490334', 91200, 91100, 91100...",NaN
2,"'000660', '2020060490334', 91200, 91100, 91200...",NaN
3,"'000660', '2020060490334', 91200, 91100, 91100...",NaN
4,"'000660', '2020060490334', 91200, 91100, 91200...",NaN
5,"'000660', '2020060490334', 91200, 91100, 91100...",NaN
...,...,...
66575,"'000660', '20200604151922', 87800, 87700, 8780...",NaN
66576,"'000660', '20200604151922', 87800, 87700, 8770...",NaN
66577,"'000660', '20200604151922', 87800, 87700, 8780...",NaN
66578,"'000660', '20200604151922', 87800, 87700, 8770...",NaN


In [ ]:
# System Trading w/ Model(MAIN LOOP)

#Instance Init
buffer = DataBuffer() # 데이터 버퍼
dataReciever = DataReciever('005930', buffer) # API에서 데이터 리시버
model = model.load(model_path)#모델 로드
logger = Logger()
trader = Trader(logger)

#완성된 모듈을 인스턴스화 하여 변수에 할당하고
#각 각의 루틴(데이터루틴, 시그널루틴/매매루틴)이 인스턴스를 참조하여 트레이딩 구현

"""
루틴1 :리시버가 API로부터 데이터를 받아올 때마다 버퍼에 (발생한)데이터를 전송하는 코드를 짜야합니당..
이 부분을 비동기 처리 해준다음(이때 버퍼의 데이터는 이 루틴에서 조작됨), 버퍼의 데이터를 받아와서
모델에 넣고 """
async def a(n):
    await asyncio.sleep(1)
    pass

async def b():
    pass

async def c():
    pass

routines = [a(3), b(), c()]

async def main():
    await asyncio.wait(routines)


if __name__ == "__main__":
    asyncio.run(main())
    

## 프로그램 한계점
#### 1개 종목에 대해서만 감시하고 매매 할 수 있게 되어있음.
#### 나중에 필요시 알고리즘을 여러개 띄우거나 여러 종목에 대해서 트레이딩하도록 구현 필요
#### 트레이딩 종목 선정 구현 안되어있음
#### 매매수량 결정에 대한 알고리즘 없음
#### 모델이 핏 되지 않음.
#### 동호가 예외처리 되지않아서 9:00-15:19 까지만 사용 가능.
#### 데이터 피처 엔지니어링을 잘 안함..